#### Model Selection

In [6]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, recall_score, precision_score, mean_squared_error

In [7]:
# Loading the pickle file Customer_Loan

with open('Pickle_Files/Credit_Card.pkl', 'rb') as f1:
     df_train_cc= pickle.load(f1)
with open('Pickle_Files/40pct.pkl', 'rb') as f2:
     df_40= pickle.load(f2)

##### Train Test Split

In [8]:
# Product - Sale
X_cc = df_train_cc.copy()
X_cc.drop(['Revenue_CC','Sale_CC'],inplace=True,axis=1)

y_sale_cc = df_train_cc['Sale_CC']

X_train_sl,X_test_sl,y_train_sl,y_test_sl = train_test_split(X_cc,y_sale_cc,test_size=0.2,random_state=0)

In [9]:
X_cc.shape

(969, 28)

In [10]:
X_train_sl.head()

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CL,ActBal_CA,...,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder
311,735,1,23,53,1,0.0,0.0,0.0,0.0,188.994643,...,339.935714,339.935714,214.285714,104.757143,16.500000,20.0,20.0,6.0,8.0,1.0
773,442,1,13,38,1,0.0,0.0,1.0,0.0,1083.015357,...,1739.750000,1739.750000,1178.571429,243.535714,311.392857,14.0,14.0,3.0,1.0,6.0
1347,955,0,19,62,1,0.0,0.0,0.0,0.0,3131.223929,...,246.528571,246.528571,0.000000,0.000000,129.642857,6.0,6.0,0.0,0.0,3.0
99,469,1,38,183,1,0.0,0.0,0.0,0.0,4415.646071,...,1549.925000,1549.925000,500.000000,0.000000,789.928571,13.0,13.0,1.0,0.0,7.0
128,536,1,29,14,1,0.0,0.0,0.0,0.0,663.843571,...,185.642857,185.642857,0.000000,0.000000,183.464286,2.0,2.0,0.0,0.0,1.0


In [11]:
# Revenue
X_cc = df_train_cc.copy()
X_cc.drop(['Revenue_CC','Sale_CC'],inplace=True,axis=1)

y_revenue_cc = df_train_cc['Revenue_CC']

X_train_rev,X_test_rev,y_train_rev,y_test_rev = train_test_split(X_cc,y_revenue_cc,test_size=0.2,random_state=0)

##### Normalize The Features

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_sl)
X_test = scaler.transform(X_test_sl)

#### Cross Validation

##### Classification

In [13]:
# Verifying which model has better score for Classification
models = {
    'KNeighbors': KNeighborsClassifier(),
    'Logistic': LogisticRegression(max_iter=10000,solver = 'sag'),
    'RandomForest': RandomForestClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
}

scores_sl = []

for model_name, model in models.items():
    # Use cross_val_score for simplicity
    cv_scores = cross_val_score(model, X_cc, y_sale_cc, cv=10)
    scores_sl.append({
        'model': model_name,
        'mean_score': cv_scores.mean(),
        'std_score': cv_scores.std(),
    })


In [14]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

scores_sl = pd.DataFrame(scores_sl)
scores_sl

,model,mean_score,std_score
0,KNeighbors,0.747090,0.044076
1,Logistic,0.749227,0.025254
2,RandomForest,0.779124,0.030190
3,DecisionTree,0.647058,0.027963


RandomForest seems to have the best score. Hence, I will be using this for model prediction. It is also more robust towards imbalanced datasets compared to KNeighbors + Logistic + DecisionTrees, as they are more biased towards majority data

In [15]:
rf = RandomForestClassifier()

rf.fit(X_train_sl,y_train_sl)

RandomForestClassifier()

In [16]:
y_pred_sl = rf.predict(X_test_sl)

##### Evaluation

In [17]:
confusion_matrix(y_pred_sl,y_test_sl)

array([[145,  37],
       [  5,   7]])

In [18]:
print('Evaluation metrics:')
print('Accuracy:', accuracy_score(y_test_sl,y_pred_sl))
print('Precision:', precision_score(y_test_sl, y_pred_sl))
print('Recall:', recall_score(y_test_sl, y_pred_sl))
print('AUC:', roc_auc_score(y_test_sl, y_pred_sl))

Evaluation metrics:
Accuracy: 0.7835051546391752
Precision: 0.5833333333333334
Recall: 0.1590909090909091
AUC: 0.562878787878788


##### Regression

In [19]:
# Define models
models = {
    'Linear': LinearRegression(),
    'RandomForest': RandomForestRegressor(),
    'DecisionTree': DecisionTreeRegressor(),
}

scores_rev = []

for model_name, model in models.items():
    # Use cross_val_score for simplicity
    cv_scores = cross_val_score(model, X_cc, y_revenue_cc, cv=10)
    scores_rev.append({
        'model': model_name,
        'mean_score': cv_scores.mean() * -1,
        'std_score': cv_scores.std(),
        
    })


In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

scores_rev = pd.DataFrame(scores_rev)
scores_rev

,model,mean_score,std_score
0,Linear,0.620661,0.888471
1,RandomForest,2.243919,2.979831
2,DecisionTree,5.180645,10.124515


I will choose RandomForest as it has a good balance between accuracy and stability

In [21]:
rf = RandomForestRegressor()

rf.fit(X_train_rev,y_train_rev)

RandomForestRegressor()

In [22]:
y_pred_rev = rf.predict(X_test_rev)

##### Evaluation

In [23]:
mse=mean_squared_error(y_test_rev,y_pred_rev)
rmse=np.sqrt(mse)

print('MSE:', mse)
print('RMSE:', rmse)


MSE: 247.5842114239969
RMSE: 15.73480890967529


#### Targeting

In [24]:
df_40_cc = df_40.drop(['Count_CC','ActBal_CC','Revenue_CL','Sale_CL','Sale_CC','Sale_MF','Revenue_CC','Revenue_MF'], axis=1)

df_40_cc.shape

(646, 28)

In [25]:
scaler.transform(df_40_cc)

array([[ 0.2148316 ,  0.95334516, -1.18006758, ..., -0.36767449,
        -0.64317598, -0.09071104],
       [-0.23555346,  0.95334516, -0.31503814, ..., -0.00560622,
        -0.64317598,  0.10456967],
       [-0.60229558, -1.04893804, -1.28819626, ...,  0.71853032,
        -0.26411789, -0.48127247],
       ...,
       [-0.73955579,  0.95334516,  1.3609564 , ..., -0.00560622,
        -0.26411789,  0.29985039],
       [-0.85751378,  0.95334516, -0.63942418, ..., -0.36767449,
        -0.51682329,  0.88569253],
       [ 1.40942436,  0.95334516,  1.0906347 , ..., -0.36767449,
        -0.51682329, -0.28599176]])

In [26]:

rev_40=rf.predict(df_40_cc)

features_df = pd.DataFrame({
    'Client':  df_40['Client'].values,
    'Revenue_CC':  rev_40
}).sort_values('Revenue_CC', ascending= False)

features_df.head(10)


,Client,Revenue_CC
90,84,133.131639
528,1073,96.356211
459,1389,83.503807
171,1460,82.448661
415,940,59.423071
343,153,56.756118
452,1162,55.036832
548,1071,52.519046
581,383,50.133504
253,1365,46.620636
